In [1]:
import numpy as np
import pickle
from keras import backend as K
from keras.models import Model, load_model
from keras.layers.recurrent_test4 import *
from model3 import *

Using TensorFlow backend.


In [50]:
print("Loading dataset...", end='')
#with open('../train_process_final.pkl', 'rb') as f:
with open('../test_process_final.pkl', 'rb') as f:
    datas = pickle.load(f)

ids, all_context, test_context, ques, encode_context, encode_ques = datas
print('Done')

Loading dataset...Done


In [58]:
print('Preparing model...')
model = RNet(vocab_size=len(embeddings_matrix), vocab_init=embeddings_matrix, hdim=75, dropout=0, p_length=300, q_length=20, char_level_embeddings=False)

model.load_weights('./models_fasttext/12-t5.969982604980469-v6.554200880468493.model')

inputs = model.inputs
outputs = [Argmax()(output) for output in model.outputs]
#outputs = [output for output in model.outputs]

predicting_model = Model(inputs, outputs)

#predicting_model.compile(optimizer='Adadelta', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

print('Done!')

print('Predicting...', end='')
#print(model.evaluate([trainP, trainQ], [train_start, train_end]))
#print(predicting_model.evaluate([valP, valQ], [val_start, val_end]))
start, end = predicting_model.predict([encode_context, encode_ques])
print('Done')

Load model...
Preparing model...__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Passage (InputLayer)            (None, 300)          0                                            
__________________________________________________________________________________________________
Question (InputLayer)           (None, 20)           0                                            
__________________________________________________________________________________________________
embedding_17 (Embedding)        multiple             12108300    Passage[0][0]                    
                                                                 Question[0][0]                   
__________________________________________________________________________________________________
bidirectional_52 (Bidirectional (None, 20, 150)      169200      embedding_17

Done!
Predicting...Done


In [64]:
num = 8
print(all_context[num])
print(ques[num])
print('======================================')
print(test_context[num][start[num]: end[num] + 1])
start_ = all_context[num].find(test_context[num][start[num]])
len_ = len(test_context[num][start[num]]) + len(test_context[num][end[num]])
print(all_context[num][start_ : start_+len_])

南宋建炎二年，為防禦金兵南下，東京守將杜充在滑州人為決開黃河堤防，造成黃河改道，向東南分由泗水和濟水入海。黃河至此由北入渤海改而南入黃海。直到1855年，黃河主要是在南面擺動，雖然時有北沖，但均被人力強行逼堵南流。
最近的一次黃河大改道是在清咸豐年間。咸豐五年六月十九日，黃河在河南蘭考北岸的銅瓦廂決口，改東北走向，在山東境內借濟水入渤海。
1938年6月9日，國軍為阻擋日軍，破壞鄭州黃河南岸花園口大堤，全河又向南流，沿賈魯河、潁河、渦河入淮河。直到1947年堵復花園口後，黃河才回歸北道。
據歷史記載，在1946年前的2540年間，黃河受到近1593次氾濫威脅，而因氾濫令河道大改道共26次。
['自', '何時', '開始', '黃河', '由北入', '渤海', '改而', '由', '南入', '黃海']
['1855', '年']
1855年


In [71]:
results = []
for index, article in enumerate(all_context):
    start_ = article.find(test_context[index][start[index]])
    len_ = len(test_context[index][start[index]]) + len(test_context[index][end[index]])
    ans_list = ' '.join([str(i) for i in range(start_, start_ + len_)])
    results.append([str(ids[index]), ans_list])

In [73]:
print(len(results))

1741


In [78]:
with open('result.txt', 'w') as f:
    f.write('id,answer\n')
    for ii in results:
        f.write(ii[0] + ',' + ii[1] + '\n')